# PET-CT Inference Tutorial
Contacts: eyuboglu@stanford.edu, gangus@stanford.edu

How to perform inference on the task of abnormality localization with a pretrained scan model.  

In this notebook we cover:  
1. Loading model configurations from a JSON like the one at `tutorials/inference/params.json`  
2. Building a `pet_ct.model.MTClassifierModel` and loading pretrained weights (Note: we do not provide pretrained weights for our models to protect PHI.)  
3. How input to the model should be structured  
4. How to perform inference on the model using   `pet_ct.model.MTClassifierModel.score`  
5. How output is structured 

## Setup
Import various packages. Make sure you're in an environment with the `pet_ct` package installed.

In [ ]:
# import requirements
%load_ext autoreload
%autoreload 2

import os
import json

import torch

import pet_ct.model.models as models
from pet_ct.model.classifier_model import MTClassifierModel
from pet_ct.learn.datasets import MTClassifierDataset
from pet_ct.learn.dataloaders import MTExamDataLoader
from pet_ct.util.util import set_logger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# TODO: change to package directory
os.chdir("/Users/sabrieyuboglu/Documents/sabri/research/projects/fdg-pet-ct/pet-ct")

experiment_dir = "tutorials/inference"
set_logger(log_path=os.path.join(experiment_dir, "process.log"))

In [ ]:
# select your CUDA devices if available
devices = []
cuda = False

## Loading hyper-parameters
We've included a params file at `notebooks/tutorial/params.json`. Please take a quick look at it toget a sense of its structure and what we include in the params".

In [ ]:
params = MTClassifierModel.load_params(os.path.join(experiment_dir, "params.json"))

# Building a model
Let's use the parameters we've loaded to build a model. We'll also load pretrained weights from `notebooks/tutorial/weights.tar`. 

In [ ]:
def build_model(model_class, model_args, weights_path=None):
    model_class = getattr(models, model_class)
    model = model_class(cuda=cuda, devices=devices, **model_args)
    if weights_path is not None: 
        model.load_weights(weights_path, device=devices[0])
    return model

In [ ]:
# build model and load weights, you should see 550/550 pretrained params loaded. 
model = build_model(params["model_class"], 
                    params["model_args"],
                    os.path.join(experiment_dir, "weights.tar"))

# How to structure inputs?
To understand how to structure inputs properly we will load some training examples from our dataset. However, the `MTClassifierDataset` class below is designed for data in our databases at Stanford. You'll likely need to write your own dataset classes for your data. You should use `MTClassifierDataset` as a template.

In [ ]:
# NOTE: this building this dataset will likely not work for you 
# because you don't have access to our data. 
# We do so here simply to demonstrate the structure of the data.
dataset = MTClassifierDataset(**params["dataset_args"], split="test")

In [ ]:
dataloader = MTExamDataLoader(dataset=dataset, 
                              num_workers=1, 
                              batch_size=1,
                              sampler="RandomSampler",
                              num_samples=200)
iterator = iter(dataloader)

Let's load an example from the dataloader and examine its structure. Each PET-CT exam is represented by a torch tensor with 4 axes. There's an additional axis for the mini-batch. Its important that your input to the model also match this structure.

In [ ]:
inputs, targets, info = iterator.next()
print(f"Input shape: {inputs.shape}")

# How to make a prediction?
Let's pass the inputs through the model using the `model.predict` function and examine the output.

In [ ]:
output = model.predict(inputs)

# How is output structured?
Let's examine what the model output. 

In [ ]:
print(f"Output is of type: {type(output)}.")
print(f"The keys of the dict are: {output.keys()}")

*Key:* The model outputs a **dictionary** with keys corresponding to each **task**.
The keys map to the predictions for the task. Let's take a look at the output for the `liver` task. 

In [ ]:
output["liver"]

Notice that the **targets** (i.e. labels) that we loaded before have a very similar structure as the output.  

In [ ]:
print(f"targets is of type: {type(targets)}.")
print(f"The keys of the dict are: {targets.keys()}")

In [ ]:
targets["liver"]

In this case the target for the liver match the output of the model. 

## How to score the model on a dataset of examples?
What if we want to evaluate the model on a dataset of examples? For this we can use the `model.score` method.

In [ ]:
metric_configs = [{'fn': 'accuracy'},
           {'fn': 'roc_auc'},
           {'fn': 'recall'},
           {'fn': 'precision'},
           {'fn': 'f1_score'}]

In [ ]:
metrics = model.score(dataloader, metric_configs=metric_configs)

We can take a look and see how the model did for this particular subset of the test set on each of the tasks. 

In [ ]:
metrics.metrics